In [1]:
import pandas as pd
import glob

In [2]:

# 여러 날짜의 데이터 파일을 불러옵니다.
file_list = glob.glob('../dataset/Milano/raw/sms-call-internet-mi-2013-11-*.txt')

# 모든 데이터를 저장할 빈 리스트를 만듭니다.
all_data = []

# 각 파일을 순회하면서 데이터를 읽어옵니다.
for file in file_list:
    data = pd.read_csv(file, sep='\t', header=None)
    data.columns = ['CellID', 'Timestamp', 'CountryCode', 'smsin', 'smsout', 'callin', 'callout', 'internet']
    all_data.append(data)

# 모든 데이터를 하나의 데이터프레임으로 합칩니다.
df = pd.concat(all_data, ignore_index=True)

# Timestamp를 datetime 형식으로 변환합니다.
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')


In [3]:
# 1. CountryCode를 제거합니다.
df = df.drop(columns=['CountryCode'])

# 2. 각 그리드의 결측치 비율을 계산하고, 결측치가 가장 적은 그리드를 선택합니다.
# 각 그리드별로 결측치가 아닌 값의 개수를 세고 가장 많은 그리드를 선택합니다.
non_null_counts = df.groupby('CellID').count()
selected_grid = non_null_counts.idxmax().max()

# 선택된 그리드의 데이터만 필터링합니다.
df_selected_grid = df[df['CellID'] == selected_grid]
# 이제 같은 그리드이므로 CellID 삭제
df_selected_grid = df_selected_grid.drop(columns=['CellID'])

# 3. 같은 그리드의 시간대를 기준으로 데이터를 합산
# Timestamp를 10분 단위로 내림합니다.
df_selected_grid['Timestamp'] = df_selected_grid['Timestamp'].dt.floor('10T')

# 그룹화 및 합산
summed_cdrs = df_selected_grid.groupby('Timestamp').sum().reset_index()

/tmp/ipykernel_1955810/3182024152.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_selected_grid['Timestamp'] = df_selected_grid['Timestamp'].dt.floor('10T')


In [4]:
summed_cdrs

,Timestamp,smsin,smsout,callin,callout,internet
0,2013-10-31 23:00:00,14.041196,10.849898,5.739816,7.912167,276.087402
1,2013-10-31 23:10:00,17.262998,10.234269,11.753134,7.848578,279.427423
2,2013-10-31 23:20:00,9.133549,3.759300,7.789001,4.465201,365.590192
3,2013-10-31 23:30:00,18.510871,9.546967,4.819154,7.026991,331.777771
4,2013-10-31 23:40:00,13.713547,5.581783,6.518861,3.491299,326.530946
...,...,...,...,...,...,...
2875,2013-11-20 22:10:00,26.827759,16.371345,9.842892,11.822882,496.213779
2876,2013-11-20 22:20:00,21.957334,10.887763,3.008837,14.590381,412.859531
2877,2013-11-20 22:30:00,13.895936,16.782342,7.185093,6.737026,375.113754
2878,2013-11-20 22:40:00,20.994752,23.917694,8.268278,11.764976,579.589388


In [5]:
# 결과 확인
print(f"선택된 그리드 ID: {selected_grid}")
print(summed_cdrs.head())

선택된 그리드 ID: 6165
            Timestamp      smsin     smsout     callin   callout    internet
0 2013-10-31 23:00:00  14.041196  10.849898   5.739816  7.912167  276.087402
1 2013-10-31 23:10:00  17.262998  10.234269  11.753134  7.848578  279.427423
2 2013-10-31 23:20:00   9.133549   3.759300   7.789001  4.465201  365.590192
3 2013-10-31 23:30:00  18.510871   9.546967   4.819154  7.026991  331.777771
4 2013-10-31 23:40:00  13.713547   5.581783   6.518861  3.491299  326.530946


In [6]:
summed_cdrs.shape

(2880, 6)

In [7]:
# 필요한 경우 결과를 CSV 파일로 저장
summed_cdrs.to_csv('../dataset/Milano/temp/preprocessed_milano.csv', index=False)

In [9]:
temp = pd.read_csv("/home/work/.default/hyunwoong/CICD_comparision/data/dataset/preprocessed_milano.csv")

In [12]:
temp

,Timestamp,smsin,smsout,callin,callout,internet
0,2013-10-31 23:00:00,14.041196,10.849898,5.739816,7.912167,276.087402
1,2013-10-31 23:10:00,17.262998,10.234269,11.753134,7.848578,279.427423
2,2013-10-31 23:20:00,9.133549,3.759300,7.789001,4.465201,365.590192
3,2013-10-31 23:30:00,18.510871,9.546967,4.819154,7.026991,331.777771
4,2013-10-31 23:40:00,13.713547,5.581783,6.518861,3.491299,326.530946
...,...,...,...,...,...,...
2875,2013-11-20 22:10:00,26.827759,16.371345,9.842892,11.822882,496.213779
2876,2013-11-20 22:20:00,21.957334,10.887763,3.008837,14.590381,412.859531
2877,2013-11-20 22:30:00,13.895936,16.782342,7.185093,6.737026,375.113754
2878,2013-11-20 22:40:00,20.994752,23.917694,8.268278,11.764976,579.589388


In [14]:
temp=temp.rename(columns={'internet': 'OT','Timestamp':'date'})

In [15]:
# 데이터프레임을 CSV 파일로 저장
temp.to_csv('/home/work/.default/hyunwoong/CICD_comparision/data/dataset/Milano.csv',index=False)